# Additional Aerial LiDAR Datasets

This notebook will highlight the additional aerial LiDAR datasets besides USGS 3dep that are available in the `coincident` package and how to grab those datasets' footprints using spatial and temporal search parameters.

Other Supported Catalogs:
- NASA G-LiHT
- NOAA Coastal LiDAR
- NCALM LiDAR
- NEON LiDAR

In [ ]:
import coincident
import geopandas as gpd

In [ ]:
%matplotlib inline
# %config InlineBackend.figure_format = 'retina'

## NASA G-LiHT

Access the NASA CMR STAC catalog for the [NASA G-LiHT project](https://gliht.gsfc.nasa.gov/about/) (Goddard LiDAR, Hyperspectral, and Thermal Imager)

In [ ]:
aoi = gpd.read_file(
    "https://raw.githubusercontent.com/unitedstates/districts/refs/heads/gh-pages/states/MD/shape.geojson"
)
aoi = aoi.simplify(0.01)
aoi.explore()

In [ ]:
gf_gliht = coincident.search.search(
    dataset="gliht",
    intersects=aoi,
    datetime=["2017"],
)
print(f"Found {len(gf_gliht)} G-LiHT Items")
print(f"G-LiHT collections: {gf_gliht.collection.unique()}")
# Select one collection, like Canopy Height (CHM) or Digital Terrain Model (DTM)
gf_gliht = gf_gliht[gf_gliht.collection == "GLCHMT_001"]
gf_gliht.explore(column="id")

## OpenTopography API

`coincident` supports the use of the [OpenTopo /otCatalog API](https://portal.opentopography.org/apidocs/) to access additional aerial LiDAR data.


opentopo datasets currently supported includes the [NOAA Coastal LiDAR Catalog](https://coast.noaa.gov/htdata/lidar1_z/) and [NCALM Aerial LiDAR Catalog](https://calm.geo.berkeley.edu/ncalm/dtc.html). 

```{note}
The NCALM Aerial LiDAR Catalog also includes user-submitted flights
```

In [ ]:
# first, we'll search for NOAA Coastal LiDAR missions in Washington
# we'll inspect 2018 arbitrarily
aoi = gpd.read_file(
    "https://raw.githubusercontent.com/unitedstates/districts/refs/heads/gh-pages/states/WA/shape.geojson"
)
date = "2018"
aoi.plot();

In [ ]:
gf_noaa = coincident.search.search(dataset="noaa", intersects=aoi, datetime=[date])

In [ ]:
gf_noaa.head(3)

In [ ]:
gf_noaa.explore(column="title", cmap="Set1")

In [ ]:
# now, let's see if there were any NCALM missions from the same year
gf_ncalm = coincident.search.search(dataset="ncalm", intersects=aoi, datetime=[date])
gf_ncalm.head()

In [ ]:
gf_ncalm.explore(column="id", cmap="Set3")

## NEON API

We also support the use of the [NEON Python API](https://www.neonscience.org/resources/learning-hub/tutorials/neon-api-intro-requests-py).

```{warning}
Searching large areas and/or large time periods in the NEON catalog will take a long time due to the limited spatiotemporal search supported by the API.
```

In [ ]:
%%time
gf_neon = coincident.search.search(dataset="neon", intersects=aoi, datetime=[date])
gf_neon.head()

In [ ]:
gf_merged = gpd.pd.concat([gf_noaa, gf_ncalm, gf_neon], ignore_index=True)
gf_merged.explore(column="collection", tiles="CartoDB positron", cmap="tab10")

## NASA LVIS

Currently, only a select number of LVIS flghts are supported due to the limited amount of LVIS products hosted on STAC catalogs:
- [ABoVE LVIS L2 Geolocated Surface Elevation Product, Version 1](https://nsidc.org/data/ablvis2/versions/1)
    - Search extent limited to bbox [-158, 48, -104, 72] and 2017-06-29 to 2017-07-17
- [AfriSAR LVIS L2 Geolocated Surface Elevation Product, Version 1](https://nsidc.org/data/aflvis2/versions/1)
    - Search extent limited to bbox [8, -2, 12, 1] and 2016-02-20 to 2016-03-08

```{warning}
Searching highly complex multi-polygons might break the STAC search, so simplifying your polygon before searching is recommended.
```

In [ ]:
# first, let's look at the ABoVE product in Canada
aoi = gpd.read_file(
    "https://gist.githubusercontent.com/M1r1k/d5731bf39e1dfda5b53b4e4c560d968d/raw/c774258085ddc11776591ce95f2240d0fd0657a2/canada_provinces.geo.json"
)
aoi = aoi[aoi["name"] == "Yukon"].reset_index(drop=True)
# reduce complexity of multipolygon input
aoi.geometry = aoi.geometry.convex_hull

In [ ]:
gf_lvis_ab = coincident.search.search(dataset="ablvis2_1", intersects=aoi)

In [ ]:
print(gf_lvis_ab.shape)
gf_lvis_ab.head(2)

In [ ]:
m = aoi.explore(color="gray")
gf_lvis_ab.clip(aoi).explore(m=m, column="datetime", cmap="inferno")

In [ ]:
# now, let's look at AfriSAR LiDAR
# this polygon is already simplified
aoi = gpd.read_file(
    "https://raw.githubusercontent.com/glynnbird/countriesgeojson/refs/heads/master/gabon.geojson"
)

In [ ]:
gf_lvis_af = coincident.search.search(
    dataset="aflvis2_1", intersects=aoi, datetime=["2016-02-20"]
)

In [ ]:
print(gf_lvis_af.shape)
gf_lvis_af.head(2)

In [ ]:
gf_lvis_af.explore(column="datetime", cmap="inferno")